<a href="https://colab.research.google.com/github/WoradeeKongthong/medical_cost_regression/blob/master/07_Medical_Cost_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
## importing the dataset

df = pd.read_csv('/content/drive/My Drive/life-long learning/MyProjects/Medical Cost/cleaned_insurance.csv',usecols=[0,1,2,3,4,5,6])

In [7]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,True,southwest,16884.92400
1,18,male,33.770,1,False,southeast,1725.55230
2,28,male,33.000,3,False,southeast,4449.46200
3,33,male,22.705,0,False,northwest,21984.47061
4,32,male,28.880,0,False,northwest,3866.85520


In [0]:
## Data Preprocessing

X = df.iloc[:, :6].values
y = df.iloc[:, 6].values

In [9]:
print(X)

[[19 'female' 27.9 0 True 'southwest']
 [18 'male' 33.77 1 False 'southeast']
 [28 'male' 33.0 3 False 'southeast']
 ...
 [18 'female' 36.85 0 False 'southeast']
 [21 'female' 25.8 0 False 'southwest']
 [61 'female' 29.07 0 True 'northwest']]


In [10]:
print(y)

[16884.924   1725.5523  4449.462  ...  1629.8335  2007.945  29141.3603]


In [0]:
# encoding categorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [1,4,5])], remainder='passthrough')
X = ct.fit_transform(X)

In [12]:
print(X[:5])

[[0.0 1.0 0.0 0.0 1.0 19 27.9 0]
 [1.0 0.0 0.0 1.0 0.0 18 33.77 1]
 [1.0 0.0 0.0 1.0 0.0 28 33.0 3]
 [1.0 0.0 1.0 0.0 0.0 33 22.705 0]
 [1.0 0.0 1.0 0.0 0.0 32 28.88 0]]


In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [26]:
# Training the Random Forest model on the Training set
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 35, max_depth=5)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=35, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [27]:
# Predicting the Test set results
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)[:10])

[[13385.68 10982.5 ]
 [34904.32 34254.05]
 [ 7285.81  7418.52]
 [ 6781.96  5594.85]
 [ 7608.74  6455.86]
 [ 5980.4   5227.99]
 [10650.91  8277.52]
 [34842.42 33475.82]
 [ 6268.07  4462.72]
 [ 6900.58  3847.67]]


In [28]:
# Model Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt
n = X_test.shape[0]
p = X_test.shape[1]
r_square = r2_score(y_test, y_pred)
adj_r_square = 1 - (1 - r_square) * ((n - 1) / (n - p - 1)) 
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mean_squared_error(y_test, y_pred))

print("R-square : ",r_square)
print("Adjusted R-square : ",adj_r_square)
print("MAE : ", mae)
print("MSE : ",mse)
print("RMSE : ",rmse)

R-square :  0.8782664676521393
Adjusted R-square :  0.8745063585448696
MAE :  2291.7526785100977
MSE :  16664282.672460321
RMSE :  4082.190915728994


In [29]:
# Applying K-Fold Cross Validation
from sklearn.model_selection import cross_val_score
y_train = y_train.reshape(len(y_train),)
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)
print("Accuracy: {}".format(accuracies.mean()))
print("Standard Deviation: {}".format(accuracies.std()))

Accuracy: 0.8506044497114795
Standard Deviation: 0.060135182024444817


In [24]:
# Improving the model

# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators':[20,25,30,35,40],'max_depth':[1,5,10,15]}]
grid_search = GridSearchCV(estimator = regressor,
                          param_grid = parameters,
                          cv = 10)
grid_search = grid_search.fit(X_train,y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy : {}".format(best_accuracy))
print("Best Parameters : ", best_parameters)

Best Accuracy : 0.861585415519744
Best Parameters :  {'max_depth': 5, 'n_estimators': 35}


In [30]:
#-----------------training evaluation--------------------------------
n = X_train.shape[0]
p = X_train.shape[1]
r_square = r2_score(y_train, regressor.predict(X_train))
adj_r_square = 1 - (1 - r_square) * ((n - 1) / (n - p - 1)) 
mae = mean_absolute_error(y_train,  regressor.predict(X_train))
mse = mean_squared_error(y_train, regressor.predict(X_train))
rmse = sqrt(mean_squared_error(y_train,  regressor.predict(X_train)))

print("R-square : ",r_square)
print("Adjusted R-square : ",adj_r_square)
print("MAE : ", mae)
print("MSE : ",mse)
print("RMSE : ",rmse)

R-square :  0.888972395628366
Adjusted R-square :  0.8881352412127458
MAE :  2304.0920488116776
MSE :  16529977.1683334
RMSE :  4065.7074622177875
